17/05/2023 au 25/05/2023    
objectif : faire la comparaison avec et sans contexte des performances du système d'entity linking de SpaCy   
contexte : notebook présenté lors de la réunion du 25/05/2023

# Evaluation comparative  du système d'entity linking de SpaCy

## code donné par CKP

In [14]:
texte = lire_fichier(path)
text=nlp(texte)  
for ent in text.ents:
        list_resultats.append([ent.text, ent.start_char, ent.end_char, ent.label_,texte[ent.start_char-200:ent.start_char],texte[ent.end_char:ent.end_char+200]])

## Première adaptation

In [3]:
import glob
import spacy
import json


def lire_fichier(chemin):
    with open (chemin, "r", encoding="utf-8") as fichier:
        texte=fichier.read()
        return texte

list_resultats=[]
path_corpora = "../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/*/*.txt"

for path in glob.glob(path_corpora):
    texte = lire_fichier(path)
    nlp = spacy.load("fr_core_news_sm")
    nlp.add_pipe("entityLinker", last=True)
    text=nlp(texte) 
    list_resultats=[]
    for ent in text.ents:
        list_resultats.append([ent.text, ent.start_char, ent.end_char, ent.label_,texte[ent.start_char-200:ent.start_char],ent.text,texte[ent.end_char:ent.end_char+200]])
print (list_resultats[0])
print (list_resultats[70])

['Norine Duclos', 76, 89, 'PER', '', " qu'ont\nlieu les plus egayantes veilles de notre vil-\nlage. Adonc, certain soir, comme j'entrais\nchez Norine, je la trouvai en train de precher\nses trois petiotes.\n- He MIandine, criait-elle, dpeche-t"]
['petiot\nRoux', 5007, 5018, 'PER', "t Jean-Claude;\nle dragon viendra pendant ce temps-la.\n- Je veux, au contraire, l'attendre pour\nchanter, dit la Rose d'un ton pique.\n- Chante, dit Gaspard, ou sinon...\n-- Tu deviens par trop bravache, ", ', dit Norine; tache un peu de te tenir\ntranquille.\n- Je vas chanter, moi, dit le vieux berger.l-\nMION VI1LAGE.\n- lEn ce cas-la, je me sauve, dit la petiote\nh Perpetue.\n- Non, dit le fieu de Toinon, so']


### Nous vérifions ensuite dans le texte si nous retrouvons bien les mêmes valeurs

"Depuis un nombre innombrable d’hivers, c’est dans la maison de Norine Duclos qu’ont lieu les plus égayantes veillées de notre village. Adonc, certain soir, comme j’entrais chez Norine, je la trouvai en train de prêcher ses trois petiotes. "

"- Chante, toi, la Rose, dit Jean-Claude;
le dragon viendra pendant ce temps-la.
- Je veux, au contraire, l'attendre pour
chanter, dit la Rose d'un ton pique.
- Chante, dit Gaspard, ou sinon...
-- Tu deviens par trop bravache, petiot
Roux, dit Norine; tache un peu de te tenir
tranquille.
- Je vas chanter, moi, dit le vieux berger.l-
MION VI1LAGE.
- lEn ce cas-la, je me sauve, dit la petiote
h Perpetue."

## Nous procédons à la NER puis appliquons l'entity linking aux entités hors contexte ...

In [9]:
import json
import spacy  # version 3.5
import glob
import matplotlib.pyplot as plt
import pandas as pd

def lire_fichier(chemin):
    with open(chemin, "r", encoding="utf-8") as fichier:
        texte = fichier.read()
        return texte

def stocker(chemin, contenu):
    with open(chemin, "w") as w:
        json.dump(contenu, w, indent=2)

modele = ["md"]
path_corpora = "../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/*/*.txt"
common_urls_per_file = []
common_urls = []
non_common_urls = []

modele=["sm","md","lg"]

for mod in modele:
    print("Pour le modèle:", mod, "\n")
    nlp = spacy.load("fr_core_news_" + mod)
    nlp.add_pipe("entityLinker", last=True)
    for path in glob.glob(path_corpora):
        print("Entrée:", path)
        texte = lire_fichier(path)
        doc = nlp(texte)
        list_resultats = []
        list_entites = []
        compteur = 0
        intersect = 0

        i = 1
        dico={}
        for ent in doc.ents:
            
            contexte = str(texte[ent.start_char-50:ent.start_char])+str(ent.text)+ str(texte[ent.end_char:ent.end_char+50])
            entite = ent.text
            print(entite)
            liste_url_entourage = []
            liste_url_entite = []
            doc2 = nlp(entite)
            doc3 = nlp(contexte)
            for ent in doc3._.linkedEntities:
                url_entourage = ent.get_url()
                liste_url_entourage.append(url_entourage)
            for ent in doc2._.linkedEntities:
                url_entite = ent.get_url()
                liste_url_entite.append(url_entite)
                for d in liste_url_entourage :
                    if d in liste_url_entite:
                        #print (d)
                        intersect = intersect + 1
            print (liste_url_entite)
            compteur +=1

Pour le modèle: md 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
VEILLÉE
[]
Norine Duclos
['https://www.wikidata.org/wiki/Q21450529']
Norine
[]
Hé
[]
Mandine
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Désirée !
['https://www.wikidata.org/wiki/Q945470']
Cellier
['https://www.wikidata.org/wiki/Q21501975']
Norine
[]
Saint-Brunelle
[]
Armandine, la Rose
['https://www.wikidata.org/wiki/Q519034']
Désirée
['https://www.wikidata.org/wiki/Q945470']
réserve de la Rose
['https://www.wikidata.org/wiki/Q519034']
Norine
[]
la Rose
['https://www.wikidata.org/wiki/Q519034']
garçonnière
['https://www.wikidata.org/wiki/Q3822506']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Roux
['https://www.wikidata.org/wiki/Q201157']
Perpétue
[]
Dieu
['https://www.wikidata.org/wiki/Q3027560']
Garde-les
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Norine
[]
Jean-Claude
['https://www.wikidat

[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Thérèse
['https://www.wikidata.org/wiki/Q3527366']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Norine
[]
Toinon
[]
Si
[]
Ah !
['https://www.wikidata.org/wiki/Q869406']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
M. Céran
['https://www.wikidata.org/wiki/Q8441', 'https://www.wikidata.org/wiki/Q859610']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Ouvrez !
[]
Mes petiots
[]
bon Dieu
['https://www.wikidata.org/wiki/Q3027560']
M. Céran
['https://www.wikidata.org/wiki/Q8441', 'https://www.wikidata.org/wiki/Q859610']
Favette
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Jeacquet
[]
Jeacquet
[]
M.
['https://www.wikidata.org/wiki/Q8441']
Jeacquet
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
madame
['https://www.wikidata.org/wiki/Q17146837']
M. Céran
['https://www.wikidata.org/wiki/Q8441', 'https://www.wikidata.org/wiki/Q859610'

[]
la Rose
['https://www.wikidata.org/wiki/Q519034']
Madame
['https://www.wikidata.org/wiki/Q17146837']
madame
['https://www.wikidata.org/wiki/Q17146837']
Norine
[]
rendez-
[]
Norine
[]
Allons
[]
la Rose
['https://www.wikidata.org/wiki/Q519034']
Norine
[]
la Rose
['https://www.wikidata.org/wiki/Q519034']
Armandine
[]
Désirée
['https://www.wikidata.org/wiki/Q945470']
Saint-Brunelle
[]
Morlincourt
['https://www.wikidata.org/wiki/Q1336814']
Rose
['https://www.wikidata.org/wiki/Q519034']
Pierre le dragon
['https://www.wikidata.org/wiki/Q40747', 'https://www.wikidata.org/wiki/Q7559']
bon Dieu
['https://www.wikidata.org/wiki/Q3027560']
Toinon
[]
Morlincourt
['https://www.wikidata.org/wiki/Q1336814']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
sou
[]
Saint-Brunelle
[]
Craintifs
[]
M. Céran
['https://www.wikidata.org/wiki/Q8441', 'https://www.wikidata.org/wiki/Q859610']
Morlinval
[]
Morlinval
[]
Céran
['https://www.wikidata.org/wiki/Q859610']
fontaine Sainte-Radegonde
['https://www.

['https://www.wikidata.org/wiki/Q201157']
Paris
['https://www.wikidata.org/wiki/Q90']
Clarisse
['https://www.wikidata.org/wiki/Q733658']
Roux
['https://www.wikidata.org/wiki/Q201157']
Clarisse
['https://www.wikidata.org/wiki/Q733658']
Palais-Royal
['https://www.wikidata.org/wiki/Q180309', 'https://www.wikidata.org/wiki/Q2102344']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Morlincourt
['https://www.wikidata.org/wiki/Q1336814']
Palais-Royal
['https://www.wikidata.org/wiki/Q180309', 'https://www.wikidata.org/wiki/Q2102344']
Morlincourt
['https://www.wikidata.org/wiki/Q1336814']
Paris
['https://www.wikidata.org/wiki/Q90']
Quoi !
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Saint-Brunelle
[]
Excusez
[]
Monsieur
['https://www.wikidata.org/wiki/Q1092777']
Parisien
[]
Monsieur
['https://www.wikidata.org/wiki/Q1092777']
Parisiens
['https://www.wikidata.org/wiki/Q3072544']
Roux
['https://www.wikidata.org/wiki/Q201157']
Sortez des maisons, gais moissonneurs
['https://www.

[]
Lasl
[]
Lasl
[]
Jeanneton
[]
long.s--------------
MION
['https://www.wikidata.org/wiki/Q16043593']
Voila
['https://www.wikidata.org/wiki/Q2351943']
renom.-

[]
Jeanneton
[]
Voila
['https://www.wikidata.org/wiki/Q2351943']
Lasl
[]
Jeanneton
[]
Lasl
[]
Jeanneton
[]
Jeanneton
[]
MION
['https://www.wikidata.org/wiki/Q16043593']
hel petiote erpctue
[]
pre Roux
['https://www.wikidata.org/wiki/Q679614', 'https://www.wikidata.org/wiki/Q201157']
Mlarie
[]
la mere Rem-
blay
['https://www.wikidata.org/wiki/Q13586859', 'https://www.wikidata.org/wiki/Q841355']
ca la religion
['https://www.wikidata.org/wiki/Q16', 'https://www.wikidata.org/wiki/Q9174']
mre
['https://www.wikidata.org/wiki/Q586615']
Je ne suis mi dcja
['https://www.wikidata.org/wiki/Q253276']
Yer-
[]
Vermand
['https://www.wikidata.org/wiki/Q994711']
l ditle
dragon
['https://www.wikidata.org/wiki/Q7559']
Pierre t
['https://www.wikidata.org/wiki/Q40747', 'https://www.wikidata.org/wiki/Q191118']
Remblay
[]
voila
['https://www.wikidata.

['https://www.wikidata.org/wiki/Q1336814']
Paris
['https://www.wikidata.org/wiki/Q90']
Morlincourt
['https://www.wikidata.org/wiki/Q1336814']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
M. Cran
['https://www.wikidata.org/wiki/Q8441', 'https://www.wikidata.org/wiki/Q5181875']
Voila
['https://www.wikidata.org/wiki/Q2351943']
Oui-dal
Claude
['https://www.wikidata.org/wiki/Q1139389']
Toinon
[]
Jean-
lMON VILLAGE
['https://www.wikidata.org/wiki/Q465227']
Claude
['https://www.wikidata.org/wiki/Q34943']
queussi-qucumi
['https://www.wikidata.org/wiki/Q180309']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Morlincourt tot
['https://www.wikidata.org/wiki/Q1336814', 'https://www.wikidata.org/wiki/Q230100']
t0t
['https://www.wikidata.org/wiki/Q191118']
M. Ceran
['https://www.wikidata.org/wiki/Q8441']
Feuil-
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Therese
['https://www.wikidata.org/wiki/Q2298877']
Ceran
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']


[]
Eee eae’i
e
e

['https://www.wikidata.org/wiki/Q704740', 'https://www.wikidata.org/wiki/Q729138']
ee
eie ee!
¬
s
[]
E. ’e.
¬
..
ee er dli e e oi e e e e ei
e &eese eq t eei eret
:
 u  iai
Ve
['https://www.wikidata.org/wiki/Q8043', 'https://www.wikidata.org/wiki/Q191118', 'https://www.wikidata.org/wiki/Q5383787', 'https://www.wikidata.org/wiki/Q9747', 'https://www.wikidata.org/wiki/Q876017']
oP:
 o 

eo
['https://www.wikidata.org/wiki/Q673', 'https://www.wikidata.org/wiki/Q204', 'https://www.wikidata.org/wiki/Q143']
Eeevirr:
ie’ ’
['https://www.wikidata.org/wiki/Q8043', 'https://www.wikidata.org/wiki/Q8043']
asl
['https://www.wikidata.org/wiki/Q14759']
S i
['https://www.wikidata.org/wiki/Q682']
ESedle ſ ie ie oei l   e e e

 
 sT 1
.p;
['https://www.wikidata.org/wiki/Q484140', 'https://www.wikidata.org/wiki/Q845104', 'https://www.wikidata.org/wiki/Q82435']
See
uie
['https://www.wikidata.org/wiki/Q55632795']
IL¬
[]
Sie pe lee lr
ieedtt
 ¬

Ee -¬
v.

['https://www.wikidata.org/wiki/Q42

['https://www.wikidata.org/wiki/Q704740', 'https://www.wikidata.org/wiki/Q17148194', 'https://www.wikidata.org/wiki/Q9887']
Ee sr’eer
-!
¬¬.3 
le e l ee lee e eee e¬
’ 
’e
['https://www.wikidata.org/wiki/Q5347002', 'https://www.wikidata.org/wiki/Q1295694', 'https://www.wikidata.org/wiki/Q180309', 'https://www.wikidata.org/wiki/Q82435', 'https://www.wikidata.org/wiki/Q487391', 'https://www.wikidata.org/wiki/Q82435', 'https://www.wikidata.org/wiki/Q704740', 'https://www.wikidata.org/wiki/Q8043', 'https://www.wikidata.org/wiki/Q8043']
Es 

2é
[]
eete ete
ei eeeoe e e
['https://www.wikidata.org/wiki/Q11621076']
i i ee e ee  t eees
 Z 4
ie ee e uie  at l ele
?
['https://www.wikidata.org/wiki/Q23809', 'https://www.wikidata.org/wiki/Q37045']
ae¬

 ¬
e e
[]
ai iei li
..
¬
e es oe e e e ie li  oee
['https://www.wikidata.org/wiki/Q347']
ee reei
Cee
['https://www.wikidata.org/wiki/Q988707']
Ee  sle i
¬
['https://www.wikidata.org/wiki/Q5347002', 'https://www.wikidata.org/wiki/Q1044', 'https://www.

['https://www.wikidata.org/wiki/Q741536']
Croyez-le
[]
honnetel
Son
[]
dte vitement regle
['https://www.wikidata.org/wiki/Q2901296']
MIais
[]
IHIommes-sans-Tetes
['https://www.wikidata.org/wiki/Q180309', 'https://www.wikidata.org/wiki/Q180309']
rabachcra
[]
ha-
sard
[]
fosses
Poulains
['https://www.wikidata.org/wiki/Q243260']
furolles
[]
LES N0CES
IY
LES NOCES
[]
Norine
[]
Jacqueline
['https://www.wikidata.org/wiki/Q391321']
Perpelue
[]
en-
fourne
[]
defourne
[]
pre Roux
['https://www.wikidata.org/wiki/Q679614', 'https://www.wikidata.org/wiki/Q201157']
honnele
[]
M0N
[]
retroussee
[]
Voila
['https://www.wikidata.org/wiki/Q2351943']
Allons  je met-

['https://www.wikidata.org/wiki/Q1045105']
Garnissons nos poches
[]
D6siree
[]
Norine
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
M0N
[]
voixt emotionnee:
- Faut
['https://www.wikidata.org/wiki/Q5438363']
quii
- Pardine
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q316479

[]
Voilh
[]
Mon pauvre fieul
[]
la France
['https://www.wikidata.org/wiki/Q142']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
meres
[]
La d6solation de la desolation
['https://www.wikidata.org/wiki/Q3815359']
h Saint-Brunellel

['https://www.wikidata.org/wiki/Q122894']
Qu'une bonne ame
['https://www.wikidata.org/wiki/Q4824411']
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
ca
[]
Cel-
lier
[]
enmoi-
meme
[]
commissio
[]
MI
['https://www.wikidata.org/wiki/Q11660']
Ceran
[]
MI0N
[]
Cran
['https://www.wikidata.org/wiki/Q5181875']
Ceran
[]
Fran-
ciers
[]
Ceran
[]
Monsieur Ceran
['https://www.wikidata.org/wiki/Q1092777']
Monsieur Ceran
['https://www.wikidata.org/wiki/Q1092777']
ouil

[]
tete
[]
Tenez
[]
Allons
[]
Courez
[]
Ceran
[]
Parlez-moi
[]
cctte bete
[]
Jean-Claude
['https://www.wikidata.org/wiki/Q3164791']
MIa-

[]
Rose
['https://www.wikidata.org/wiki/Q519034']
Ceran
[]
The-
rese
[]
unl
[]
Ceran
[]
madame
['https://www.wikidata.org/wiki/Q17146837']
Ceran
[]
je1
-

M

## Nous comparons ensuite les URLs obtenues lors du linking pour les entités en contexte et hors contexte

In [11]:
import json
import spacy  # version 3.5
import glob
import pandas as pd

def lire_fichier(chemin):
    with open(chemin, "r", encoding="utf-8") as fichier:
        texte = fichier.read()
        return texte
    
def tableau(data):
    # Création du tableau récapitulatif
    df = pd.DataFrame(data)
    print (df)
    table_latex = df.to_latex(index=False)
    print(table_latex)

modele=["sm","md","lg"]
path_corpora = "../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/*/*.txt"
donnee = []

for mod in modele:
    print("Pour le modèle:", mod, "\n")
    nlp = spacy.load("fr_core_news_" + mod)
    nlp.add_pipe("entityLinker", last=True)
    for path in glob.glob(path_corpora):
        print("Entrée:", path)
        texte = lire_fichier(path)
        doc = nlp(texte)
        compteur = 0
        intersect = 0
        nb_entite = 0

        for ent in doc.ents:
            contexte = str(texte[ent.start_char-50:ent.start_char])+str(ent.text)+ str(texte[ent.end_char:ent.end_char+50])
            entite = ent.text
            
            liste_url_entourage = []
            liste_url_entite = []
            doc3 = nlp(entite)
            doc2 = nlp(contexte)
            for ent in doc2._.linkedEntities:
                url_entourage = ent.get_url()
                liste_url_entourage.append(url_entourage)
            for ent in doc3._.linkedEntities:
                url_entite = ent.get_url()
                liste_url_entite.append(url_entite)
                for d in liste_url_entite:
                    if d in liste_url_entourage:
                        intersect += 1
                    compteur += 1
            nb_entite += 1
        
        non = compteur - intersect
        print("La REN de Spacy a identifié", nb_entite, "entités")
        print("L'entity linker a repéré", compteur, "entités")
        print("Il y a", intersect, "éléments communs")
        print("Il y a", non, "éléments qui ne sont pas en commun")

        donnee.append({
            "Modèle": mod,
            "Fichier": path,
            "Entités reconnues par Spacy": nb_entite,
            "Entités reconnues par l'Entity Linker": compteur,
            "Entités avec URLs communes ": intersect,
            "Entités avec URLs non communes": non
        })
tableau(donnee)

Pour le modèle: md 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
La REN de Spacy a identifié 790 entités
L'entity linker a repéré 653 entités
Il y a 602 éléments communs
Il y a 51 éléments qui ne sont pas en commun
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
La REN de Spacy a identifié 1256 entités
L'entity linker a repéré 1072 entités
Il y a 817 éléments communs
Il y a 255 éléments qui ne sont pas en commun
  Modèle                                            Fichier  \
0     md  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
1     md  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OC...   

   Entités reconnues par Spacy  Entités reconnues par l'Entity Linker  \
0                          790                                    653   
1                         1256                                   1072   

   Entités avec URLs communes   Entités avec URLs non communes  
0                          602 

/tmp/ipykernel_3388/539984079.py:15: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table_latex = df.to_latex(index=False)


## Nous avons étudié le nombre d'URL associé à chaque entité

In [12]:
import spacy 
import glob
import pandas as pd

def lire_fichier(chemin):
    with open(chemin, "r", encoding="utf-8") as fichier:
        texte = fichier.read()
        return texte
def tableau(data):
    # Création du tableau récapitulatif
    df = pd.DataFrame(data)
    print (df)
    table_latex = df.to_latex(index=False)
    print(table_latex)

modele=["sm","md","lg"]

path_corpora = "../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/*/*.txt"
data = []
tab_entite = []
for mod in modele:
    print("Pour le modèle:", mod, "\n")
    nlp = spacy.load("fr_core_news_" + mod)
    nlp.add_pipe("entityLinker", last=True)
    for path in glob.glob(path_corpora):
        print("Entrée:", path)
        texte = lire_fichier(path)
        doc = nlp(texte)
        compteur = 0
        intersect = 0
        nb_entite = 0

        for ent in doc.ents:
            contexte = str(texte[ent.start_char-50:ent.start_char])+str(ent.text)+ str(texte[ent.end_char:ent.end_char+50])
            entite = ent.text
            #print ("pour l'entité", entite)
            liste_url_entourage = []
            liste_url_entite = []
            doc3 = nlp(entite)
            doc2 = nlp(contexte)
            for ent in doc2._.linkedEntities:
                url_entourage = ent.get_url()
                liste_url_entourage.append(url_entourage)
            for ent in doc3._.linkedEntities:
                url_entite = ent.get_url()
                liste_url_entite.append(url_entite)
                #print("Il y a",len(liste_url_entite),"url correspondantes. Ce sont", liste_url_entite)
                tab_entite.append({
                    "Modèle": mod,
                    "Fichier": path,
                    "Entité reconnue par la REN de SpaCy": entite,
                    "nb d'entités repérées par l'Entity Linker pour cette entité": len(liste_url_entite),
                    "url(s) liée(s) par l'Entity Linker pour cette entité": liste_url_entite,
                })

            for d in liste_url_entite:
                if d in liste_url_entourage:
                    intersect += 1
                compteur += 1
            nb_entite += 1
        
        non = compteur - intersect
        #print("La REN de Spacy a identifié", nb_entite, "entités")
        #print("L'entity linker a repéré", compteur, "entités")
        #print("Il y a", intersect, "éléments communs")
        #print("Il y a", non, "éléments qui ne sont pas en communs")
        
        data.append({
            "Modèle": mod,
            "Fichier": path,
            "Entités Spacy": nb_entite,
            "Entités Entity Linker": compteur,
            "Entités communes": intersect,
            "Entités non communes": non
        })
        
        
tableau(tab_entite)


Pour le modèle: sm 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
Pour le modèle: md 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
Pour le modèle: lg 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
     Modèle                                            Fichier  \
0        sm  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
1        sm  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
2        sm  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
3        sm  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
4        sm  ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/RE...   
...     ...                    

/tmp/ipykernel_3388/2682745239.py:13: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  table_latex = df.to_latex(index=False)


\begin{tabular}{lllrl}
\toprule
Modèle &                                            Fichier &                Entité reconnue par la REN de SpaCy &  nb d'entités repérées par l'Entity Linker pour cette entité & url(s) liée(s) par l'Entity Linker pour cette entité \\
\midrule
    sm & ../DATA/ELTeC-Fra\_2023/ADAM/ADAM\_krakenbase/REF... &                                      Norine Duclos &                                                  1 &          [https://www.wikidata.org/wiki/Q21450529] \\
    sm & ../DATA/ELTeC-Fra\_2023/ADAM/ADAM\_krakenbase/REF... &                                        Jean-Claude &                                                  1 &           [https://www.wikidata.org/wiki/Q3164791] \\
    sm & ../DATA/ELTeC-Fra\_2023/ADAM/ADAM\_krakenbase/REF... &                                               Rose &                                                  1 &            [https://www.wikidata.org/wiki/Q519034] \\
    sm & ../DATA/ELTeC-Fra\_2023/ADAM/ADAM\_krakenbas

## Entités qui n'ont pas les mêmes URLs liées en contexte et hors contexte

In [ ]:
import spacy  # version 3.5
import glob
import pandas as pd
import os

def lire_fichier(chemin):
    with open(chemin, "r", encoding="utf-8") as fichier:
        texte = fichier.read()
        return texte
def tableau(data):
    # Création du tableau récapitulatif
    df = pd.DataFrame(data)
    print (df)
    table_latex = df.to_latex(index=False)
    print(table_latex)

modele=["fr_core_news_sm","fr_core_news_md","fr_core_news_lg", "en_core_web_sm", "en_core_web_md", "en_core_web_lg"]


path_corpora = "../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/*/*.txt"
donnee = []
tab_entite = []
tab_noncommun = []
tab_basename = []

for mod in modele:
    print("Pour le modèle:", mod, "\n")
    nlp = spacy.load(mod)
    nlp.add_pipe("entityLinker", last=True)
    for path in glob.glob(path_corpora):
        basename = os.path.basename(path)
        if basename in tab_basename :
            pass
        else :
            tab_basename.append(basename)
        print("Entrée:", path)
        texte = lire_fichier(path)
        
        
        doc = nlp(texte)
        compteur = 0
        intersect = 0
        nb_entite = 0
        

        for ent in doc.ents:
            contexte = str(texte[ent.start_char-50:ent.start_char])+str(ent.text)+ str(texte[ent.end_char:ent.end_char+50])
            entite = ent.text
            #print ("pour l'entité", entite)
            liste_url_entourage = []
            liste_url_entite = []
            liste_label_entourage = []
            liste_label_entite = []
            liste_description_entourage = []
            liste_description_entite = []
            doc3 = nlp(entite)
            doc2 = nlp(contexte)
            for ent in doc2._.linkedEntities:
                url_entourage = ent.get_url()
                label_entourage = ent.get_label()
                description_entourage = ent.get_description()
                liste_url_entourage.append(url_entourage)
                liste_label_entourage.append(label_entourage)
                liste_description_entourage.append (description_entourage)
                liste_description_entite = []
                concat_entourage = nom_entourage," : ",description_entourage
                #print (concat_entourage)
                liste_concat_entourage.append(concat_entourage)
            for ent in doc3._.linkedEntities:
                url_entite = ent.get_url()
                nom_entite = ent.get_label()
                description_entite = ent.get_description()
                concat_entite = str(nom_entite) + " : " + description_entite
                #print (concat_entite)
                liste_url_entite.append(url_entite)
                liste_concat_entite.append(concat_entite)
                #print("Il y a",len(liste_url_entite),"url correspondantes. Ce sont", liste_url_entite)
                tab_entite.append({
                    "Modèle": mod,
                    "Fichier": basename,
                    "Entité reconnue par la REN de SpaCy": entite,
                    "nb d'entités repérées par l'Entity Linker": len(liste_url_entite),
                    "Entités reconnues par l'Entity Linker": liste_concat_entite,
                })

            for d in liste_url_entite:
                if d in liste_url_entourage:
                    intersect += 1
                else :
                    tab_noncommun.append({
                    "Modèle": mod,
                    "Fichier": basename,
                    "Entité reconnue par la REN de SpaCy": entite,
                    "entités reconnues pour l'Entity Linker sans contexte": liste_concat_entite,
                    "entités reconnues pour l'Entity Linker en contexte": liste_concat_entourage,
                })
                compteur += 1
            nb_entite += 1
        
        non = compteur - intersect
        #print("La REN de Spacy a identifié", nb_entite, "entités")
        #print("L'entity linker a repéré", compteur, "entités")
        #print("Il y a", intersect, "éléments communs")
        #print("Il y a", non, "éléments qui ne sont pas en communs")
        
        donnee.append({
            "Modèle": mod,
            "Fichier": basename,
            "Entités Spacy": nb_entite,
            "Entités Entity Linker": compteur,
            "Entités communes": intersect,
            "Entités non communes": non
        })
tableau(tab_entite)
#tableau(donnee)
#tableau(tab_noncommun)


Pour le modèle: fr_core_news_sm 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
Pour le modèle: fr_core_news_md 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
Pour le modèle: fr_core_news_lg 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
Pour le modèle: en_core_web_sm 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/OCR/ADAM_Mon-village_Kraken-base.txt
Pour le modèle: en_core_web_md 

Entrée: ../DATA/ELTeC-Fra_2023/ADAM/ADAM_krakenbase/REF/ADAM_Mon-village_REF.txt
